<h1> Import Libraries and Stuff </h1>

In [1]:
#import libraries and modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from random import randint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from NeuralNetwork import Layer, NeuralNetwork
from PSO import ParticleStructure, Swarm
import itertools
import time

#supress all warnings
import warnings
warnings.filterwarnings('ignore')

#import data to pandas dataframe
df = pd.read_csv('iris.csv')



<h1> Data Preprocessing </h1>

<h3>This is a rough segment </h3>

In [ ]:
# #import data to pandas dataframe
# df = pd.read_csv('iris.csv')

# #convert dataframe to numpy array
# data = df.values

# #change all the categorical data to numeric data
# data[data == 'setosa'] = 0
# data[data == 'versicolor'] = 1
# data[data == 'virginica'] = 2

# #data splitting into training and test
# X_tv, X_test, y_tv, y_test = train_test_split(data[:,0:4], data[:,4], test_size = 0.2, random_state = 42)
# X_train, X_validation, y_train, y_validation = train_test_split(X_tv, y_tv, test_size = 0.16, random_state = 40)

# #dimension fix nd reshape
# X_train = X_train.astype(float).T
# X_validation = X_validation.astype(float).T
# X_test = X_test.astype(float).T

# y_train = y_train.astype(float).T
# y_train = np.reshape(y_train , (1, 100))

# y_validation = y_validation.astype(float).T
# y_validation = np.reshape(y_validation , (1, 20))

# y_test = y_test.astype(float).T
# y_test = np.reshape(y_test , (1, 30))

# #y_train data transformation
# y_train_c = np.zeros((3, 100))
# for i in range(y_train.shape[1]):
#     if y_train[0][i]==0:
#         y_train_c[0][i] = 1
#     elif y_train[0][i]==1:
#         y_train_c[1][i] = 1
#     else:
#         y_train_c[2][i] = 1
# y_train = y_train_c

# #y_validation data transformation
# y_validation_c = np.zeros((3, 20))
# for i in range(y_validation.shape[1]):
#     if y_validation[0][i]==0:
#         y_validation_c[0][i] = 1
#     elif y_validation[0][i]==1:
#         y_validation_c[1][i] = 1
#     else:
#         y_validation_c[2][i] = 1

# #y_test data transformation
# y_test_c = np.zeros((3, 30))
# for i in range(y_test.shape[1]):
#     if y_test[0][i]==0:
#         y_test_c[0][i] = 1
#     elif y_test[0][i]==1:
#         y_test_c[1][i] = 1
#     else:
#         y_test_c[2][i] = 1

# #function body for prediction mapping ([1,0,0]->0->Setosa)
# def prediction_mapping(predict):
#     predict = np.round(predict, 2)
#     predict[predict<0.5] = 0
#     predict[predict>=0.5] = 1

#     y_predict = np.zeros((1, 30))

#     for i in range (predict.shape[1]):
#         col = [row[i] for row in predict]
#         if col == [1.0, 0.0, 0.0]:
#             y_predict[0][i] = 0
#         elif col == [0.0, 1.0, 0.0]:
#             y_predict[0][i] = 1
#         else:
#             y_predict[0][i] = 2
#     return y_predict

<h3> Main Data Preprocessing </h3>

In [2]:
#convert dataframe to numpy array
data = df.values

#change all the categorical data to numeric data
data[data == 'setosa'] = 0
data[data == 'versicolor'] = 1
data[data == 'virginica'] = 2

#data splitting into training and test
X_train, X_test, y_train, y_test = train_test_split(data[:,0:4], data[:,4], test_size = 0.2, random_state = 42)

#dimension fix nd reshape
X_train = X_train.astype(float).T
X_test = X_test.astype(float).T
y_train = y_train.astype(float).T
y_train = np.reshape(y_train , (1, 120))
y_test = y_test.astype(float).T
y_test = np.reshape(y_test , (1, 30))

#data transformation
y_train_c = np.zeros((3, 120))

for i in range(y_train.shape[1]):
    if y_train[0][i]==0:
        y_train_c[0][i] = 1
    elif y_train[0][i]==1:
        y_train_c[1][i] = 1
    else:
        y_train_c[2][i] = 1

y_train = y_train_c

y_test_c = np.zeros((3, 30))

for i in range(y_test.shape[1]):
    if y_test[0][i]==0:
        y_test_c[0][i] = 1
    elif y_test[0][i]==1:
        y_test_c[1][i] = 1
    else:
        y_test_c[2][i] = 1
        
X_test.shape


(4, 30)

<h1> ES Implementation </h1>

In [11]:
#######################################
# Variables
#######################################

#selection parameter
mu = 5
pop_size = 20              #number of child networks (this is \lamda)
pop_parent = []            #parent network hyperparameter container

activation_inside = ['ReLU', 'tanh', 'sigmoid']
activation_bool = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
taboo = [] #add restriction in random selection       
 
#######################################
# Functions
#######################################

# parent initialize in random with restriction of uniqueness
def parent_initialize_in_restriction(chromosome):
    np.random.seed(chromosome)            #initialize all random with fixed seed
    new_candidate = [np.random.randint(4,33), activation_bool[np.random.randint(0,3)]]
    if new_candidate in taboo:
        return "In Taboo"
    else:
        return new_candidate
    #return parent_initialize_in_restriction(chromosome) if new_candidate in taboo else new_candidate

# here we will be selecting top \mu number of parents
# we are basically applying truncation selection

def parentSelection(pop_parent, pop_final_loss):
    fittest_parents = []
    sorted_indices = np.argsort(pop_final_loss, axis=0)
    for i in range(mu):
        fittest_parents.append(pop_parent[sorted_indices[i][0]])
    return fittest_parents

#mutation operator for the first element
def mutationA(selectionA, selected_parents, i, max_bound, min_bound):
    #The Toss for A
    tossA = random.random()
    copy_A = selected_parents[i][0]
    if tossA > selectionA: #selection pressure 30%
        #selected_parents[i][0] = np.random.randint(4,33)
        copy_A = copy_A + (-1)**np.random.randint(1,3)
    return copy_A
        
#mutation operator for the second element
def mutationB(selectionB, selected_parents, i):
    #The Toss for B
    tossB = random.random()
    copy_B = selected_parents[i][1]
    if tossB > selectionB: #selection pressure 50%
        #Implement bitwise rightshift in boolean vector
        index = selected_parents[i][1].index(1)
        copy_B = [0,0,0]
        index = index + 1
        if index > 2:
            index = 0
        copy_B[index] = 1
        #selected_parents[i][1] = copy
    return copy_B                

def breeding(selected_parents):
    max_bound = 33
    min_bound = 4
    pop_parent = []    #empty current population
    #generate lamda children
    for i in range(len(selected_parents)):
        #inner loop control
        for j in range(int(pop_size/mu)):
            #mutation for A
            copyA = mutationA(0.3, selected_parents, i, max_bound, min_bound)
            #mutation for B
            copyB = mutationB(0.5, selected_parents, i)    
            #new child candidate
            new_child = [copyA, copyB]
            #check in taboo list
            check = 1
            while check <= 5:
                if new_child in taboo:   #ensure that no child can be in taboo list
                    copyA = mutationA(0.3, selected_parents, i, max_bound, min_bound)
                    copyB = mutationB(0.5, selected_parents, i)    
                    new_child = [copyA, copyB]
                    check = check + 1
                else:
                    break
            taboo.pop(0)
            taboo.append(new_child)
            pop_parent.append(new_child)
            print(new_child)
        

####################################
# Step 1: Population Initialization
####################################
c_check = 1 #being used to avoid generating duplicate values already in taboo list from using same seed
for chromosome in range(pop_size):
    new_candidate = parent_initialize_in_restriction(c_check)
    while (new_candidate=="In Taboo"):
        c_check = c_check + 1
        new_candidate = parent_initialize_in_restriction(c_check)
    taboo.append(new_candidate)
    pop_parent.append(new_candidate)
    print(new_candidate)

#########################################
# Step 2: ES Loop for fitness assessment
#########################################

best_chromosome = np.zeros((1, 2))
best_loss = 0
round_num = 1
global_best_chromosome = np.zeros((1,2))
global_best_loss = 0

while round_num <=10:
    print("---------------------------------------------")
    print("Generation ", round_num, " Population:")
    print("---------------------------------------------")
    pop_final_loss = np.zeros((pop_size, 1))
    for i in range(pop_size):

        ############################################
        # Step 3: Individual Fitness Assessment
        ############################################

        # Neural Network Structure Define
        number_of_hidden_layers = 2
        number_of_input_layer_units = X_train.shape[0]
        number_of_hidden_layer_units = pop_parent[i][0]
        number_of_output_layer_units = np.size(pop_parent[i][1])#change chromosome->i
        number_of_samples = X_train.shape[1]
        hidden_layer_activation = pop_parent[i][1]
        P_Net = NeuralNetwork([number_of_hidden_layers, number_of_input_layer_units, number_of_hidden_layer_units, \
                           number_of_output_layer_units, number_of_samples, hidden_layer_activation])
        P_Net.structure_define()
        
        #one single forward pass for fitness
        P_Net.forward_propagation(X_train)
        loss = P_Net.calculate_network_loss(y_train)
        
        # Loss calculation
        pop_final_loss[i] = loss

        #save the best candidate
        if i == 0:
            best_chromosome = pop_parent[i]
            best_loss = pop_final_loss[i]
        elif pop_final_loss[i] < best_loss:
            best_chromosome = pop_parent[i]
            best_loss = pop_final_loss[i]
    
    if round_num==1:
        global_best_loss = best_loss
        global_best_chromosome = best_chromosome
    elif best_loss < global_best_loss:
        global_best_loss = best_loss
        global_best_chromosome = best_chromosome

    print ("Best Loss of Generation: ", round_num, "= ", best_loss)
    print ("Best Chromosome of Generation: ", round_num, "= ", best_chromosome)
    print ("Global Best Loss after Generation: ", round_num, "= ", global_best_loss)
    print ("Global Best Chromosome after Generation: ", round_num, "= ", global_best_chromosome)
    print("---------------------------------------------")
    ############################################
    # Step 4: Truncation Selection
    ############################################

    selected_parents = parentSelection(pop_parent, pop_final_loss) #select \mu fittest parents 
    #print(selected_parents)

    ############################################
    # Step 5: Population Regeneration
    ############################################

    if round_num < 10: 
        breeding(selected_parents)
    round_num = round_num + 1


    



[9, [1, 0, 0]]
[12, [0, 1, 0]]
[14, [1, 0, 0]]
[30, [0, 0, 1]]
[7, [0, 0, 1]]
[14, [0, 1, 0]]
[19, [1, 0, 0]]
[7, [1, 0, 0]]
[32, [0, 0, 1]]
[13, [0, 1, 0]]
[29, [1, 0, 0]]
[15, [0, 0, 1]]
[22, [1, 0, 0]]
[15, [1, 0, 0]]
[13, [0, 0, 1]]
[19, [0, 1, 0]]
[18, [0, 1, 0]]
[13, [1, 0, 0]]
[25, [1, 0, 0]]
[23, [0, 0, 1]]
---------------------------------------------
Generation  1  Population:
---------------------------------------------
Best Loss of Generation:  1 =  [1.97067427]
Best Chromosome of Generation:  1 =  [7, [0, 0, 1]]
Global Best Loss after Generation:  1 =  [1.97067427]
Global Best Chromosome after Generation:  1 =  [7, [0, 0, 1]]
---------------------------------------------
[6, [1, 0, 0]]
[8, [0, 0, 1]]
[7, [0, 0, 1]]
[6, [0, 0, 1]]
[21, [0, 1, 0]]
[22, [0, 1, 0]]
[23, [1, 0, 0]]
[21, [1, 0, 0]]
[14, [0, 0, 1]]
[14, [0, 1, 0]]
[13, [0, 1, 0]]
[15, [0, 1, 0]]
[11, [0, 0, 1]]
[12, [0, 1, 0]]
[11, [0, 1, 0]]
[12, [0, 0, 1]]
[30, [1, 0, 0]]
[29, [1, 0, 0]]
[31, [1, 0, 0]]
[31, [